Ostatecznie w rozwiązaniu zdecydowałem się umieścić regresyjną maszynę wektorów wspierających. Dlaczego zdecydowałem się na ten model? Po prostu lubię ten model i dla tych danych mógł się okazać całkiem dobrym modelem. Ostatecznie zdecydowałem się na umieszczenie tego modelu w tej wersji, która na leaderboardzie dała mi wynik 0.21465. Jest to najlepszy wynik, który uzyskałem dla parametrów gamma = 0.1, C = 1 oraz kernel = linear trenując na całym zbiorze treningowym. Jednak gdy ten sam model trenowałem na 75%, by sprawdzić uzyskany błąd walidacyjny, to błąd walidacyjny okazał się niezadowalający, a na dodatek wynik na leaderboardzie wyniósł powyżej 0.3. Podobnie było, gdy dla tych parametrów wprowadziłem jeszcze kroswalidacje z podziałami na 5 i 7 podzbiorów. Próbowałem też inne parametry, jak np. kernel = rbf i przy okazji regulowanie gammy oraz różne kroswalidacje, jednak nie dało to oczekiwanych rezultatów. Poza maszyną wektorów wspierających spróbowałem modeli Elastic Net oraz Random Forest z zadań 2 i 3, jednak wyniki były zdecydowanie poniżej oczekiwań. 

Rozważyłem też maszynę wektorów wspierających, dla której zdefiniowałem siatkę złożoną z parametrów: C - parametr regularyzacji: {0.1, 1, 10, 100, 1000}, gamma - parametr dla RBF: {1, 0.1, 0.01, 0.001, 0.0001}, kernel: {'rbf', 'linear'}. Był to zdecydowany rywal dla wybranego modelu i bardzo długo zastanawiałem się, czy go nie wybrać. Niestety zapomniałem sprawdzić, jakie parametry wybrała siatka, a model się liczył całą noc, więc już nie zdążyłem puścić go ponownie, co z kolei ograniczyło mocno ewentualne poprawianie wybranego przez siatkę modelu. Dla tego modelu dostałem wynik 0.26448, czyli istotnie gorszy od wyniku uzyskanego dla tego modelu, natomiast jest to wynik mimo wszystko całkiem dobry, a rozwiązanie wydaje się dużo pewniejsze i bezpieczniejsze. Uznałem jednak, że mimo że model, dla którego uzyskałem najlepszy wynik nie jest pewny, tzn. na pozostałych 70% zbioru może zawieść, to postanowiłem zaryzykować i postawiłem na ten model.

Regresyjna maszyna wektorów wspierających to model, w którym minimalizujemy funkcję
$$J(\beta) = \frac{1}{2} \beta'\beta + C \sum_{n=1}^N(\xi_n + \xi_n^*)$$
przy warunkach
$$\forall_n: y_n - (x_n'\beta + b) \leq \epsilon + \xi_n$$
$$\forall_n: (x_n'\beta + b) - y_n \leq \epsilon + \xi_n^*$$
$$\forall_n: \xi_n \geq 0$$
$$\forall_n: \xi_n^*\geq 0$$
$C$ jest pozytywną wartością regularyzującą model

In [1]:
#importujemy potrzebne biblioteki
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler
#from sklearn.feature_selection import SelectKBest
#from sklearn.feature_selection import chi2
from sklearn import svm
#from sklearn.model_selection import train_test_split
#from sklearn.metrics import balanced_accuracy_score
#from sklearn.feature_selection import RFE

Wczytuję dane:

In [2]:
X_train = pd.read_csv("X_train.csv")
y_train = pd.read_csv("y_train.csv")
X_test = pd.read_csv("X_test.csv")

Skaluję dane:

In [3]:
min_on_training = X_train.min(axis=0) 
range_on_training = (X_train - min_on_training).max(axis=0)
X_train_scaled = (X_train - min_on_training) / range_on_training
X_test_scaled = (X_test - min_on_training) / range_on_training

Definiuję i trenuję model:

In [4]:
clf = svm.SVR(kernel='linear', C=1, gamma=0.1)

In [5]:
clf.fit(X_train_scaled, y_train['Expected'])

SVR(C=1, gamma=0.1, kernel='linear')

Predykcje:

In [6]:
y_pred = clf.predict(X_test_scaled)

In [7]:
y_pred

array([0.96359768, 1.8009981 , 6.96395311, 1.98571098, 1.28706772,
       0.77481968, 4.74292683, 7.27470432, 1.92084502, 1.68443822,
       6.96014424, 1.06033131, 1.46648622, 1.19028158, 0.77026219,
       0.64994597, 1.19832661, 2.22141006, 1.27117822, 2.74037047,
       1.58465262, 2.81843598, 1.84519885, 1.64256531, 6.9936589 ,
       1.59108213, 1.72010682, 1.0766288 , 1.77146003, 1.03365566,
       1.81001377, 7.11066262, 6.92773527, 1.40432486, 7.2506964 ,
       1.51998636, 1.31189029, 1.47368836, 1.24347113, 6.9820157 ,
       1.79822207, 6.93763837, 5.7946165 , 6.64533612, 6.82385654,
       1.09708668, 1.38686513, 0.79067098, 6.7811328 , 6.99145475,
       0.27453284, 0.94306187, 1.39621908, 1.10947301, 1.43368659,
       2.77138919, 5.84891306, 1.1492667 , 6.65909646, 1.82960276,
       7.01369776, 3.69790918, 1.67249977, 1.79802882, 0.91475002,
       1.3190424 , 6.95218722, 7.0040139 , 1.31158713, 1.39401397,
       6.76635422, 1.08648023, 1.89564189, 6.92962757, 0.51900

Przygotowanie danych do zapisania do pliku:

In [8]:
data = {
  "Id": np.arange(0,670),
  "Expected": y_pred
}

In [9]:
SVM = pd.DataFrame(data)

In [10]:
SVM

,Id,Expected
0,0,0.963598
1,1,1.800998
2,2,6.963953
3,3,1.985711
4,4,1.287068
...,...,...
665,665,1.074892
666,666,1.339070
667,667,6.777162
668,668,1.050850


In [11]:
SVM.to_csv("SVM.csv")